# Analysis Reference PEP

## Setup and Helper Functions

### Imports

In [ ]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import seaborn as sns
from biopsykit.stats import StatsPipeline
from fau_colors import cmaps, register_fausans_font

from pepbench.data_handling import (
    correlation_reference_pep_heart_rate,
    describe_pep_values,
    get_reference_data,
    rr_interval_to_heart_rate,
)
from pepbench.export import convert_to_latex, create_reference_pep_table
from pepbench.io import load_challenge_results_from_folder
from pepbench.plotting.results import boxplot_reference_pep, regplot_pep_heart_rate, histplot_heart_rate

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties_light)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

In [ ]:
root_path = Path("../../")

In [ ]:
deploy_type = "local"

config_dict = json.load(root_path.joinpath("config.json").open(encoding="utf-8"))

empkins_base_path = Path(config_dict[deploy_type]["empkins_path"])
guardian_base_path = Path(config_dict[deploy_type]["guardian_path"])

In [ ]:
paper_path = json.load(root_path.joinpath("paper_path.json").open(encoding="utf-8"))["paper_path"]
paper_path = Path(paper_path)

result_path = root_path.joinpath("results")
export_path = root_path.joinpath("exports")
img_path = export_path.joinpath("plots")
stats_path = export_path.joinpath("stats")

img_path_paper = paper_path.joinpath("img")
tab_path_paper = paper_path.joinpath("tab")
suppl_img_path_paper = paper_path.joinpath("supplementary_material/img")
suppl_tab_path_paper = paper_path.joinpath("supplementary_material/tab")

bp.utils.file_handling.mkdirs([result_path, export_path, img_path, stats_path, img_path_paper, tab_path_paper, suppl_img_path_paper, suppl_tab_path_paper])

In [ ]:
algo_levels = ["q_peak_algorithm", "b_point_algorithm", "outlier_correction_algorithm"]

## EmpkinS Dataset

In [ ]:
result_dict_empkins = {}

In [ ]:
condition_mapping_empkins = {"tsst": "TSST", "ftsst": "f-TSST"}
phase_mapping_empkins = {
    "Prep": "Preparation",
    "Pause_1": "Pause 1",
    "Talk": "Talk",
    "Math": "Math",
    "Pause_5": "Pause 5",
}

phase_order_empkins = list(phase_mapping_empkins.values())

In [ ]:
results_empkins = load_challenge_results_from_folder(
    result_path.joinpath("empkins_dataset_both_algorithms"), index_cols_per_sample=["participant", "condition", "phase"]
)

In [ ]:
results_empkins.per_sample

In [ ]:
reference_data_empkins = get_reference_data(results_empkins.per_sample)
reference_data_empkins

In [ ]:
reference_pep_empkins = reference_data_empkins[["pep_ms"]]
# reference_pep_empkins = reference_pep_empkins.reindex(results_empkins.per_sample.index.get_level_values("phase").unique(), level="phase")
reference_pep_empkins

In [ ]:
pep_describe_total_empkins = reference_pep_empkins.agg(["mean", "std", "min", "max"]).round(2).T
result_dict_empkins["pep_describe_total"] = pep_describe_total_empkins

pep_describe_total_empkins

In [ ]:
hr_describe_total_empkins = reference_data_empkins[["heart_rate_bpm"]].agg(["mean", "std", "min", "max"]).round(2).T
result_dict_empkins["hr_describe_total"] = hr_describe_total_empkins

hr_describe_total_empkins

In [ ]:
reference_data_empkins_rename = reference_data_empkins.rename(condition_mapping_empkins, level="condition").rename(
    phase_mapping_empkins, level="phase"
)

reference_pep_empkins_rename = reference_pep_empkins.rename(condition_mapping_empkins, level="condition").rename(
    phase_mapping_empkins, level="phase"
)

In [ ]:
pep_describe_empkins = describe_pep_values(reference_pep_empkins_rename, group_cols=["condition", "phase"])
pep_describe_empkins = pep_describe_empkins.reindex(phase_order_empkins, level="phase")

result_dict_empkins["describe_pep_phases"] = pep_describe_empkins

pep_describe_empkins

In [ ]:
result_table = create_reference_pep_table(
    pep_describe_empkins,
)

latex_output = convert_to_latex(
    result_table,
    collapse_index_columns=False,
    column_header_bold=True,
    position="h!",
    label="tab:reference_pep_empkins",
    caption=r"Summary of reference \ac{PEP} values for the different conditions and phases of the "
    r"\textit{EmpkinS Dataset}. The range is provided as [min, max].",
    column_format="p{2.0cm}p{2.0cm}S[table-format=3.3(5)]p{2.0cm}",
)
# fix pandas bug that does not format the last column name in bold
latex_output = latex_output.replace(r"{Range", r"{\bfseries Range")

suppl_tab_path_paper.joinpath("tab_reference_pep_empkins.tex").open(mode="w+").write(latex_output)

print(latex_output)

In [ ]:
steps = [("test", "rm_anova"), ("posthoc", "pairwise_tests")]
params = {
    "dv": "pep_ms",
    "within": ["condition", "phase"],
    "subject": "participant",
    "parametric": False,
    "multicomp": {"levels": True, "method": "bonf"},
}

pipeline_empkins = StatsPipeline(
    steps=steps,
    params=params,
)

pipeline_empkins.apply(reference_pep_empkins_rename)
pipeline_empkins.display_results()
pipeline_empkins.export_statistics(stats_path.joinpath("stats_reference_pep_empkins.xlsx"))

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

box_pairs, pvalues = pipeline_empkins.sig_brackets(
    stats_category_or_data="posthoc", stats_effect_type="interaction", plot_type="single"
)

boxplot_reference_pep(
    reference_pep_empkins_rename,
    x="condition",
    y="pep_ms",
    hue="phase",
    width=0.90,
    stats_kwargs={"box_pairs": box_pairs, "pvalues": pvalues, "verbose": 0},
    legend_loc="lower center",
    legend_orientation="horizontal",
    legend_fontsize="small",
    title="EmpkinS Dataset",
    rect=(0, 0.05, 1, 1),
    ax=ax,
)
ax.set_xlim(-0.5, 1.5)

fig.savefig(img_path.joinpath("img_reference_pep_empkins.pdf"), transparent=True)

### Heart Rate

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4))

hr_empkins = reference_data_empkins_rename[["heart_rate_bpm"]]
fig, ax = histplot_heart_rate(data=hr_empkins, hue="phase", stat="percent", kde=True, ax=ax)

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_hr_distribution_empkins.pdf"), transparent=True)

In [ ]:
fig, ax = regplot_pep_heart_rate(results_empkins.per_sample, use_reference=True, add_corr_coeff=True, figsize=(7, 4))
corr_results = correlation_reference_pep_heart_rate(results_empkins.per_sample)

result_dict_empkins["pep_hr_correlation"] = corr_results["correlation"]
result_dict_empkins["pep_hr_linear_regression"] = corr_results["linear_regression"]

display(corr_results["correlation"])
display(corr_results["linear_regression"])

ax.set_title("EmpkinS Dataset", fontweight="bold")

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_reference_pep_hr_empkins.pdf"), transparent=True)

In [ ]:
bp.io.write_pandas_dict_excel(result_dict_empkins, export_path.joinpath("result_summary_empkins.xlsx"), index_col=True)

## Guardian Dataset

In [ ]:
result_dict_guardian = {}

In [ ]:
phase_mapping_guardian = {
    "Pause": "Pause",
    "Valsalva": "Valsalva",
    "HoldingBreath": "Holding Breath",
    "TiltUp": "Tilt Up",
    "TiltDown": "Tilt Down",
}
phase_order_guardian = list(phase_mapping_guardian.keys())

In [ ]:
results_guardian = load_challenge_results_from_folder(
    result_path.joinpath("guardian_dataset_both_algorithms"), index_cols_per_sample=["participant", "phase"]
)

In [ ]:
results_guardian.per_sample

In [ ]:
reference_data_guardian = get_reference_data(results_guardian.per_sample)
reference_data_guardian

In [ ]:
reference_pep_guardian = reference_data_guardian[["pep_ms"]]
reference_pep_guardian

In [ ]:
pep_describe_total_guardian = reference_pep_guardian.agg(["mean", "std", "min", "max"]).round(2).T
result_dict_guardian["pep_describe_total"] = pep_describe_total_guardian

pep_describe_total_guardian

In [ ]:
hr_describe_total_guardian = reference_data_guardian[["heart_rate_bpm"]].agg(["mean", "std", "min", "max"]).round(2).T
result_dict_guardian["hr_describe_total"] = hr_describe_total_guardian

hr_describe_total_guardian

In [ ]:
pep_describe_guardian = describe_pep_values(reference_pep_guardian, group_cols=["phase"])
pep_describe_guardian = pep_describe_guardian.reindex(phase_order_guardian, level="phase")
result_dict_guardian["pep_describe_phases"] = pep_describe_guardian

pep_describe_guardian

In [ ]:
result_table = create_reference_pep_table(pep_describe_guardian)

latex_output = convert_to_latex(
    result_table,
    collapse_index_columns=False,
    column_header_bold=True,
    position="h!",
    label="tab:reference_pep_guardian",
    caption=r"Summary of reference \ac{PEP} values for the different phases of the "
    r"\textit{Guardian Dataset}. The range is provided as [min, max].",
    column_format="p{2.0cm}S[table-format=3.3(5)]p{2.0cm}",
)
# fix pandas bug that does not format the last column name in bold
latex_output = latex_output.replace(r"{Range", r"{\bfseries Range")

suppl_tab_path_paper.joinpath("tab_reference_pep_guardian.tex").open(mode="w+").write(latex_output)

print(latex_output)

In [ ]:
reference_data_guardian_rename = reference_data_guardian.rename(phase_mapping_guardian, level="phase")
reference_pep_guardian_rename = reference_pep_guardian.rename(phase_mapping_guardian, level="phase")

In [ ]:
steps = [("test", "rm_anova"), ("posthoc", "pairwise_tests")]
params = {
    "dv": "pep_ms",
    "within": "phase",
    "subject": "participant",
    "parametric": False,
    "multicomp": {"levels": True, "method": "bonf"},
}

pipeline_guardian = StatsPipeline(
    steps=steps,
    params=params,
)

pipeline_guardian.apply(reference_pep_guardian_rename)
pipeline_guardian.display_results()
pipeline_empkins.export_statistics(stats_path.joinpath("stats_reference_pep_guardian.xlsx"))

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

box_pairs, pvalues = pipeline_guardian.sig_brackets(
    stats_category_or_data="posthoc", stats_effect_type="within", plot_type="single"
)

boxplot_reference_pep(
    reference_pep_guardian_rename,
    x="phase",
    y="pep_ms",
    stats_kwargs={"box_pairs": box_pairs, "pvalues": pvalues, "verbose": 0},
    rect=(0, 0.05, 1, 1),
    title="Guardian Dataset",
    ax=ax,
)

fig.savefig(img_path.joinpath("img_reference_pep_guardian.pdf"), transparent=True)

### Heart Rate

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4))

hr_guardian = reference_data_guardian_rename[["heart_rate_bpm"]]
fig, ax = histplot_heart_rate(data=hr_guardian, hue="phase", stat="percent", kde=True, ax=ax)

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_hr_distribution_guardian.pdf"), transparent=True)

In [ ]:
fig, ax = regplot_pep_heart_rate(results_guardian.per_sample, use_reference=True, add_corr_coeff=True, figsize=(7, 4))

corr_results = correlation_reference_pep_heart_rate(results_guardian.per_sample)

result_dict_guardian["pep_hr_correlation"] = corr_results["correlation"]
result_dict_guardian["pep_hr_linear_regression"] = corr_results["linear_regression"]

display(corr_results["correlation"])
display(corr_results["linear_regression"])

ax.set_title("Guardian Dataset", fontweight="bold")

for path in [img_path, suppl_img_path_paper]:
    fig.savefig(path.joinpath("img_reference_pep_hr_guardian.pdf"), transparent=True)

## Combined Figure

In [ ]:
fig, axs = plt.subplots(figsize=(10, 4), ncols=2, sharey=True)

box_pairs, pvalues = pipeline_empkins.sig_brackets(
    stats_category_or_data="posthoc", stats_effect_type="interaction", plot_type="single"
)

boxplot_reference_pep(
    reference_pep_empkins_rename,
    x="condition",
    y="pep_ms",
    hue="phase",
    showmeans=True,
    width=0.95,
    stats_kwargs={"box_pairs": box_pairs, "pvalues": pvalues, "verbose": 0, "text_offset": -2.5},
    show_legend=False,
    title="EmpkinS Dataset – Reference PEP",
    rect=(0, 0.05, 1, 1),
    ax=axs[0],
)
axs[0].set_xlim(-0.5, 1.5)

handles, labels = axs[0].get_legend_handles_labels()
axs[0].legend().remove()
axs[0].legend(handles, labels, ncols=3, fontsize="small")

box_pairs, pvalues = pipeline_guardian.sig_brackets(
    stats_category_or_data="posthoc", stats_effect_type="within", plot_type="single"
)

boxplot_reference_pep(
    reference_pep_guardian_rename,
    x="phase",
    y="pep_ms",
    showmeans=True,
    width=0.90,
    stats_kwargs={"box_pairs": box_pairs, "pvalues": pvalues, "verbose": 0, "text_offset": -2.5},
    rect=(0, 0.05, 1, 1),
    title="Guardian Dataset – Reference PEP",
    ax=axs[1],
)
axs[1].set_ylabel(None)

fig.tight_layout()

for path in [img_path, img_path_paper]:
    fig.savefig(path.joinpath("img_reference_pep_combined.pdf"), transparent=True)

## Export

In [ ]:
bp.io.write_pandas_dict_excel(
    result_dict_guardian, export_path.joinpath("result_summary_guardian.xlsx"), index_col=True
)